In [3]:
import pandas as pd
import numpy as np
import requests
import re
import os
from time import sleep
from tqdm import tqdm
from functools import reduce

import datetime

In [13]:
BASE_GOOGLE_FINANCE_URL = "https://api.polygon.io/v2/aggs/ticker/{TICKER}/range/{INTERVAL}/" +\
                          "{RANGE}/{FROM_DATE}/{TO_DATE}?limit=50000&apiKey=MNxlbyNXPpKmAki1jcScqmXxzPvdXWYT"

TOKENS = [
    "AAPL", "GOOG", "MSFT", "TSLA", "NVDA", "AMZN",
    "FB", "BABA", "CRM", "INTC", "AMD", "PYPL", "ATVI",
    "EA", "TTD", "MTCH", "ZG", "YELP"
]
# TOKENS = ["AAPL"]

TIME_INTERVAL = 15
TIME_UNITS = "minute"
FROM_DATE = pd.Timestamp("2019-02-18").date()
TO_DATE = pd.Timestamp("2022-04-04").date()

DATASET_NAME = "tech_companies"

FULL_RESULT_PATH = f"prices/{DATASET_NAME}_{TIME_INTERVAL}{TIME_UNITS}.csv"
TRAIN_RESULT_PATH = f"prices/{DATASET_NAME}_{TIME_INTERVAL}{TIME_UNITS}_train.csv"
TEST_RESULT_PATH = f"prices/{DATASET_NAME}_{TIME_INTERVAL}{TIME_UNITS}_test.csv"

In [5]:
def get_url(token: str, interval_len: int, interval: str, from_date: datetime.date, to_date: datetime.date):
    """
    Build url for Google Finance API
    """
    url = re.sub("{TICKER}", token, BASE_GOOGLE_FINANCE_URL)
    url = re.sub("{INTERVAL}", str(interval_len), url)
    url = re.sub("{RANGE}", interval, url)
    url = re.sub("{FROM_DATE}", str(from_date), url)
    url = re.sub("{TO_DATE}", str(to_date), url)
    return url

In [6]:
def get_data(token: str, interval_len: int, interval: str, from_date: datetime.date, to_date: datetime.date):
    TIMEOUT = 20
    current_date = from_date
    previous_date = current_date
    dfs = []
    while current_date <= to_date:
        request_url = get_url(token, interval_len, interval, current_date, to_date)
        data = requests.get(url=request_url)
        if data.json()["status"] == "ERROR":
            print("Waiting for", TIMEOUT, "seconds")
            for _ in tqdm(range(TIMEOUT)):
                sleep(1)
        else:
            try:
                current_df = pd.DataFrame(data.json()["results"])
            except:
                pass
            current_date = (pd.Timestamp(current_df["t"].max(), unit="ms") + pd.DateOffset(days=1)).date()
            print("Current date:", current_date)
            print("Get DataFrame of shape", current_df.shape)
            dfs.append(current_df)
    result_df = pd.concat(dfs)
    result_df.drop_duplicates(inplace=True)
    return result_df

In [7]:
dfs = []
for token in TOKENS:
    data = get_data(
        token=token,
        interval_len=TIME_INTERVAL,
        interval=TIME_UNITS,
        from_date=FROM_DATE, 
        to_date=TO_DATE
    )

    data["t"] = data["t"].apply(lambda row: pd.Timestamp(row, unit="ms"))
    data["ticker"] = token
    dfs.append(data)

Current date: 2020-07-15
Get DataFrame of shape (4395, 8)
Current date: 2020-10-08
Get DataFrame of shape (3809, 8)
Current date: 2021-01-01
Get DataFrame of shape (3662, 8)
Current date: 2021-03-26
Get DataFrame of shape (3616, 8)
Current date: 2021-06-24
Get DataFrame of shape (3939, 8)
Waiting for 20 seconds


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:20<00:00,  1.01s/it]


Waiting for 20 seconds


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:20<00:00,  1.01s/it]


Current date: 2021-09-24
Get DataFrame of shape (4041, 8)
Current date: 2021-12-22
Get DataFrame of shape (3909, 8)
Current date: 2022-03-22
Get DataFrame of shape (3853, 8)
Current date: 2022-04-05
Get DataFrame of shape (639, 8)
Current date: 2020-09-30
Get DataFrame of shape (4582, 8)
Waiting for 20 seconds


...

Current date: 2021-10-08
Get DataFrame of shape (3938, 8)
Current date: 2022-04-05
Get DataFrame of shape (3852, 8)
Current date: 2020-10-29
Get DataFrame of shape (4358, 8)
Current date: 2021-06-05
Get DataFrame of shape (4666, 8)
Current date: 2022-01-04
Get DataFrame of shape (4701, 8)
Waiting for 20 seconds


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:20<00:00,  1.01s/it]


Waiting for 20 seconds


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:20<00:00,  1.01s/it]


Current date: 2022-04-05
Get DataFrame of shape (2010, 8)
Current date: 2020-10-14
Get DataFrame of shape (4268, 8)
Current date: 2021-05-04
Get DataFrame of shape (4244, 8)
Current date: 2021-12-04
Get DataFrame of shape (4265, 8)
Current date: 2022-04-05
Get DataFrame of shape (2328, 8)


In [12]:
RENAME_COLUMNS = {
    "v": "volume",
    "o": "open",
    "c": "close",
    "h": "high",
    "l": "low",
    "t": "date"
}
KEEP_COLUMNS = ["volume", "open", "close", "high", "low", "date", "ticker"]

result_df = pd.concat(dfs).rename(RENAME_COLUMNS, axis=1)[KEEP_COLUMNS].reset_index(drop=True)
print(result_df.shape)
result_df.sample(10)

(445787, 7)


,volume,open,close,high,low,date,ticker
66309,11548.0,257.7600,257.8100,258.0000,257.7600,2021-04-23 11:30:00,MSFT
431204,15967.0,20.8200,20.7550,20.8200,20.7550,2020-04-28 18:05:00,YELP
36119,20508.0,1535.3500,1532.7600,1535.8400,1532.0000,2020-09-16 17:15:00,GOOG
442225,13143.0,39.7500,39.7500,39.7500,39.7500,2021-10-01 20:00:00,YELP
250968,1306.0,210.0100,209.9500,210.0100,209.9500,2022-03-03 13:30:00,CRM
311728,4209779.0,112.0800,112.6532,112.7000,111.3400,2022-02-24 19:00:00,AMD
266994,474621.0,66.2085,66.3899,66.4700,66.1750,2021-04-07 18:30:00,INTC
416161,30683.0,58.6600,58.7000,58.9000,58.6000,2020-06-02 16:04:00,ZG
43456,17493.0,2629.8000,2626.2500,2630.5000,2625.5200,2021-07-13 14:30:00,GOOG
160264,94052.0,3499.6800,3500.4150,3507.2099,3499.0701,2021-09-02 14:45:00,AMZN


In [14]:
TEST_RATIO = 0.2
test_size = int(result_df.shape[0] * TEST_RATIO)
data_train = result_df.iloc[:-test_size]
data_test = result_df.iloc[-test_size:]

result_df.to_csv(FULL_RESULT_PATH)
data_train.to_csv(TRAIN_RESULT_PATH)
data_test.to_csv(TEST_RESULT_PATH)